In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,35762
2,Huelva,Confirmados PDIA 14 días,1831
3,Huelva,Tasa PDIA 14 días,"356,8018395463492"
4,Huelva,Confirmados PDIA 7 días,1118
5,Huelva,Total Confirmados,35955
6,Huelva,Curados,32826
7,Huelva,Fallecidos,395


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  35762.0
/tmp/ipykernel_18250/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  9929.0
/tmp/ipykernel_18250/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

/tmp/ipykernel_18250/2690025379.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_18250/2690025379.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_18250/2690025379.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 328 personas en los últimos 7 días 

Un positivo PCR cada 220 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,35762.0,1118.0,1831.0,513170.0,217.861527,356.801840,506.0
Huelva-Costa,20897.0,798.0,1308.0,289548.0,275.601973,451.738572,350.0
Huelva (capital),9929.0,439.0,653.0,143837.0,305.206588,453.986109,186.0
Condado-Campiña,11410.0,232.0,397.0,156231.0,148.498057,254.110900,110.0
Sierra de Huelva-Andévalo Central,3082.0,85.0,111.0,67391.0,126.129602,164.710421,44.0
Aljaraque,1288.0,63.0,95.0,21474.0,293.378039,442.395455,31.0
Ayamonte,1491.0,59.0,103.0,21104.0,279.567854,488.059136,31.0
Isla Cristina,2441.0,77.0,184.0,21393.0,359.930818,860.094423,30.0
Moguer,1525.0,60.0,89.0,21867.0,274.386061,407.005991,26.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Niebla,309.0,12.0,45.0,4116.0,291.545190,1093.294461,3.0
Isla Cristina,2441.0,77.0,184.0,21393.0,359.930818,860.094423,30.0
Santa Bárbara de Casa,37.0,2.0,8.0,1043.0,191.754554,767.018217,NaN
Cortelazor,15.0,2.0,2.0,299.0,668.896321,668.896321,NaN
Palos de la Frontera,803.0,47.0,63.0,11742.0,400.272526,536.535514,24.0
Ayamonte,1491.0,59.0,103.0,21104.0,279.567854,488.059136,31.0
Zarza-Perrunal (La),36.0,3.0,6.0,1253.0,239.425379,478.850758,2.0
Lepe,2342.0,59.0,131.0,27880.0,211.621234,469.870875,23.0
Huelva (capital),9929.0,439.0,653.0,143837.0,305.206588,453.986109,186.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Alosno,209.0,2.0,8.0,3933.0,50.851767,203.407068,1.0,0.250000
Santa Bárbara de Casa,37.0,2.0,8.0,1043.0,191.754554,767.018217,0.0,0.250000
Niebla,309.0,12.0,45.0,4116.0,291.545190,1093.294461,3.0,0.266667
Rosal de la Frontera,37.0,1.0,3.0,1697.0,58.927519,176.782557,0.0,0.333333
San Juan del Puerto,616.0,7.0,19.0,9411.0,74.381043,201.891404,6.0,0.368421
Gibraleón,699.0,11.0,28.0,12737.0,86.362566,219.831986,6.0,0.392857
Isla Cristina,2441.0,77.0,184.0,21393.0,359.930818,860.094423,30.0,0.418478
Palma del Condado (La),1232.0,13.0,29.0,10801.0,120.359226,268.493658,7.0,0.448276
Lepe,2342.0,59.0,131.0,27880.0,211.621234,469.870875,23.0,0.450382
